In [ ]:
###### Solo usar para la replica de los modelos - considerados.

In [1]:
!pip install shap
!pip install scorecardpy
!pip install sweetviz

     |████████████████████████████████| 564 kB 14.9 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.2-py3-none-any.whl size=57843 sha256=1d08f4215247914bfe360402eeaf8ca3a3d72b0b99ff282dd13c45a01e07da49
  Stored in directory: /root/.cache/pip/wheels/9d/50/52/715367c478fd2e2317070ccf6539b2c29fc8168144cfbfaeac
Successfully built scorecardpy
     |████████████████████████████████| 15.1 MB 23.8 MB/s 


In [ ]:
import numpy as np
import itertools
import pandas as pd
import gc
import datetime
import sweetviz as sv
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
from scipy import stats
from seaborn import violinplot
import seaborn as sns
import shap
import scorecardpy as sc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgbm
from matplotlib import pyplot
from pylab import rcParams
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
##  Cargando la data de testeo.
test=pd.read_csv('rpp-ds-challenge/ds_challenge_2021.csv')

In [ ]:
print(test.shape)

In [ ]:
test.describe()

In [ ]:
## Identificando registros con las mismas caracteristicas, recordando que cada registro son los diferentes movimientos o transacciones de diferentes clientes.
Duplicados=test.iloc[:,1:].duplicated()
print(Duplicados.any())

## No hay duplicados.

In [ ]:
## Convertiendo la variable "Fraude" y "is_prime" a variables enteras.
# Data_set.fraude=Data_set.fraude.replace({'False':0,'True':1})
# Data_set['fraude']=Data_set['fraude'].astype(int)
test['is_prime']=test['is_prime'].astype(int)

In [ ]:
test['Dia'] =pd.to_datetime(test['fecha']).dt.day_name()

del test['fecha']

In [ ]:
### para la muestra test, tbm.
test['genero']=np.where(test['genero']=='--','Error registro',test['genero'])

In [ ]:
# Variables a observar:
vars_cat = ['establecimiento','ciudad','genero']

In [ ]:
### aplicando lo mismo para la muestra test
for i in vars_cat:
    print("---------------------------------------------------------------------------------------------")
    test[i]=test[i].replace(np.nan,"No registrado") 

In [ ]:
### lo mismo para el test.

test["Sistema"]="."
test.loc[test['dispositivo'].str.contains('ANDROID'),"Sistema"] = 'ANDROID'
test.loc[test['dispositivo'].str.contains('WEB'), "Sistema"] = 'WEB'
test.loc[test['dispositivo'].str.contains('%%'),"Sistema"] = '%%'

test["device_score"]=""
test.loc[test['dispositivo'].str.contains('1,'),"device_score"] = '1'
test.loc[test['dispositivo'].str.contains('2,'), "device_score"] = '2'
test.loc[test['dispositivo'].str.contains('3,'),"device_score"] = '3'
test.loc[test['dispositivo'].str.contains('4,'),"device_score"] = '4'
test.loc[test['dispositivo'].str.contains('5,'),"device_score"] = '5'

In [ ]:
del test['dispositivo']

In [ ]:
#### Lo mismo se considerara para la muestra Test.
test['Flag_dcto']=np.where(test['dcto']>0,1,0)
del test['dcto']

In [ ]:
### llamando a los datos de entrenamiento, ya que se necesitan los valores WOE para aplicarlos en la nueva muestra.
train=pd.read_csv('rpp-ds-challenge/train_EDA.csv')
del train['Unnamed: 0']

In [ ]:
#### Guaramos los cortes definidos por iv y conversando con el negocio.
breaks_adj={'Dia': ['Friday%,%Sunday%,%Saturday','Monday%,%Wednesday%,%Thursday%,%Tuesday'], 'Flag_dcto': [1.0], 'Sistema': ['.%,%WEB%,%%%','ANDROID'], 'cashback': [4.0,7.0,9.0,15.0], 'ciudad': ['Monterrey','Merida','Toluca','No registrado%,%Guadalajara'], 'device_score': ['1','2%,%5%,%3%,%4' ], 'establecimiento': ['Restaurante','Farmacia','Abarrotes%,%Super','MPago%,%No registrado'], 'genero': ['M','F%,%Error registro'], 'hora': [4.0,7.0,20.0,22.0], 'interes_tc': [39.0,41.0,57.0,63.0], 'is_prime': [1.0], 'linea_tc': [73000.0,77000.0,83000.0,90000.0], 'monto': [70.0,130.0,260.0,420.0], 'status_txn': ['Rechazada','Aceptada','En proceso'], 'tipo_tc': ['Virtual','Física']}

In [ ]:
bins_adj = sc.woebin(train, y="fraude", breaks_list=breaks_adj) 

#### Aplicando los valores WOE obtenidos con la muestra de entrenamiento.

test_woe = sc.woebin_ply(test, bins_adj) 

In [ ]:
###### Se eleminan las variables menos importantes (Variables se identificarón en el entrenamiento)

In [ ]:
#   list(Variables_menos_importantes)
Variables_menos_importantes=['genero_woe','establecimiento_woe','ciudad_woe','device_score_woe','Dia_woe','tipo_tc_woe','Sistema_woe','status_txn_woe','is_prime_woe','Flag_dcto_woe']

In [ ]:
### para la muestra test.
for i in list(Variables_menos_importantes):
    del test_woe[i]

# **Cargamos los modelos guardados.**

Modelo Logit

In [ ]:
import joblib

fraude_log= joblib.load('modelo_logit.joblib')

In [ ]:
#### Obtenemos la predicción con el modelo - logit.

rf_pred_class_test = fraude_log.predict(test_woe.iloc[:, 1:])
rf_probs_test = fraude_log.predict_proba(test_woe.iloc[:, 1:])

In [ ]:
test['prob']=rf_probs_test